<a href="https://colab.research.google.com/github/gzromerof/consultas_bd_sunat/blob/main/multisectores2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Matriz_MC es la matriz que contiene a las empresas ya clasificadas por Tamaño
import pandas as pd
df= pd.read_excel('/content/drive/MyDrive/Colab Notebooks/Adex/Base_Ene_MC2.xlsx', sheet_name='Base_Ene_MC')
df

,AA,MM,NUM_DOCUMENTO,DECLARANTE,NOMB_CONTINENTE_ES,COD_PAIS,NOMB_PAIS_ES,NOMB_VIA_ES,SECTOR_ES,PARTIDA,DESC_ARANCELARIA_ES,DPTO,FOB,Tamaño 2021,Tamaño 2022,Tamaño 2023
0,2021,1,20100003512,INDUSTRIAS ELECTRO QUIMICAS S.A.,AFRICA,ZA,"SUDAFRICA, REPUBLICA DE",MARITIMO,SIDERURGICO Y METALURGICO,7905000091,"DISCOS, HEXAGONOS,DE CINC CUYA MAYOR DIMENSION...",CALLAO,82855.50,Grande,Grande,Grande
1,2021,1,20100003512,INDUSTRIAS ELECTRO QUIMICAS S.A.,AMERICA DEL SUR,EC,ECUADOR,MARITIMO,SIDERURGICO Y METALURGICO,7901200000,ALEACIONES DE CINC,CALLAO,28056.00,Grande,Grande,Grande
2,2021,1,20100003512,INDUSTRIAS ELECTRO QUIMICAS S.A.,EUROPA,DE,ALEMANIA,MARITIMO,QUIMICO,2817001000,OXIDO DE CINC (BLANCO O FLOR DE CINC),CALLAO,324907.25,Grande,Grande,Grande
3,2021,1,20100003512,INDUSTRIAS ELECTRO QUIMICAS S.A.,EUROPA,FR,FRANCIA,MARITIMO,MINERIA TRADICIONAL,7901110000,"CINC SIN ALEAR, CON UN CONTENIDO DE CINC SUPER...",CALLAO,34089.79,Grande,Grande,Grande
4,2021,1,20100003512,INDUSTRIAS ELECTRO QUIMICAS S.A.,EUROPA,FR,FRANCIA,MARITIMO,SIDERURGICO Y METALURGICO,7905000012,LAMINADOS PLANOS DE CINC DE ESPESOR SUPERIOR A...,CALLAO,167129.69,Grande,Grande,Grande
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
355719,2023,6,Ley 29733 P,NaN,EUROPA,NL,PAISES BAJOS,MARITIMO,VARIOS,9503009300,LOS DEMAS JUQGUETES QUE REPRESENTEN ANIMALES O...,LIMA,6766.50,Grande,Grande,Grande
355720,2023,6,Ley 29733 P,NaN,EUROPA,PL,POLONIA,AEREA,VARIOS,7113190000,ART. DE JOYERIA DE LOS DEMAS METALES PRECIOSOS...,LIMA,529.00,Grande,Grande,Grande
355721,2023,6,Ley 29733 P,NaN,OCEANIA,AU,AUSTRALIA,AEREA,METAL - MECANICO,7326909000,LAS DEMAS MANUFACTURAS DE HIERRO O ACERO,LIMA,600.00,Grande,Grande,Grande
355722,2023,6,Ley 29733 P,NaN,OCEANIA,AU,AUSTRALIA,AEREA,PRENDAS DE VESTIR,6110301000,"SUETERES, PULLOVERS, CARDIGANS, CHALECOS Y ART...",LIMA,7200.00,Grande,Grande,Grande


In [ ]:
def multisectores(df, anio: int=None, tipo=0):

    """
    Muestra el número de empresas exportadoras por sector económico.
    :df: DataFrame.
    :anio: Último año del dataframe (opcional, por defecto año actual).
    :tipo: Tipo de exportación (0: anual acumulado, 1: mes específico).
    :return: None
    """

    if tipo not in (0, 1):
      print("Error: el argumento 'tipo' solo acepta los valores 0 (acumulado anual) o 1 (mensual).")
      return

    #Si no se declara argumento 'anio', se utiliza, por defecto, el año actual.
    min_AA= min(df["AA"].unique())
    max_AA= max(df["AA"].unique())
    if anio is None:
      anio = max_AA
    elif not isinstance(anio, int) or anio<min_AA or anio>max_AA:
      print(f"Error: en este caso, a partir del análisis de la base de datos, el argumento 'anio' solo acepta valores enteros, desde {min_AA} hasta {max_AA}.")
      return

    #Determina cuántos años (máx. 10) se analizarán contando a partir del último año del dataframe.
    intentos2=4
    while True:
      intentos2-=1
      if intentos2==0:
        print("Excedió el límite de intentos.")
        return
      try:
        n = int(input(f"Ingrese el número de años a analizar, contando desde el argumento 'anio' (en este caso: {anio}): "))
        if 1<=n<=(anio - min_AA + 1):
          break # Sale del bucle si n es válido
        else:
          print(f'En este caso, a partir del análisis de la base de datos, el número de años a analizar debe ser menor o igual a {anio-min_AA+1}.')
      except ValueError:
        print('Error: el número de años a analizar debe ser un número entero.')

    #Creamos diccionario vacío para almacenar "SUB" DataFrames filtrados por año.
    df_dict = {}
    if tipo==0 :
      for i in range(0,n):
        df_dict[f'df{anio-i}'] = df[df['AA'] == anio-i]
    elif tipo==1 :
      intentos3=4
      min_MM=min(df["MM"].unique())
      max_MM=max(df["MM"].unique())
      while True:
        intentos3-=1
        if intentos3==0:
          print("Excedió el límite de intetos.")
          return
        try:
          mes = int(input(f"Ingrese el mes [{min_MM}, {max_MM}] (a partir del análisis de la base de datos): "))
          if min_MM <= mes <= max_MM:
              break  # Salir del bucle si el mes es válido
          else:
              print(f'En este caso, a partir del análisis de la base de datos, el mes debe ser un número entero, desde {min_MM} hasta {max_MM}.')
        except ValueError:
          print('Error: el mes debe ser un número entero.')

      for i in range(0,n):
          df_dict[f'df{anio-i}'] = df[(df['AA'] == anio-i) & (df['MM']==mes)]

    #Aplicación de filtros al DataFrame.
    import pandas as pd
    #Creamos diccinario vacío para almacenar Dataframes filtrados por año y FOB.
    multisectores_dict = {}
    for i in range(0,n):
      #Agrupa por empresa, país y tamaño, y calcula la suma de ventas.
      f_multisectores = df_dict[f'df{anio-i}'].groupby(
          ['NUM_DOCUMENTO', 'SECTOR_ES', f'Tamaño {anio-i}'], as_index=False).agg({'FOB': 'sum'})
      #Calcula las ventas por empresa utilizando el DataFrame filtrado original.
      ventas_por_empresa = df_dict[f'df{anio-i}'].groupby('NUM_DOCUMENTO')['FOB'].sum().reset_index()
      ventas_por_empresa = ventas_por_empresa.rename(columns={'FOB': 'VentaTotal'})
      #Fusiona los DataFrames para combinar las ventas por empresa.
      f_multisectores = pd.merge(f_multisectores, ventas_por_empresa, on='NUM_DOCUMENTO')
      #Selecciona las columnas necesarias
      f_multisectores = f_multisectores[['NUM_DOCUMENTO', 'SECTOR_ES', f'Tamaño {anio-i}', 'FOB', 'VentaTotal']]
      f_multisectores.reset_index(drop=True, inplace=True)
      #Filtra los datos de empresas cuyas ventas totales sean >=1000 y, a su vez, que sus ventas por país >=1000
      #Nota: supongamos que una empresa exporta $1000: $500 Argentina y $500 Chile, entonces, esta empresa se considerará
      #en el conteo de empresas exportadoras (>=1000), PERO, NO se considerará en los sectores Argentina y Chile.
      f_multisectores = f_multisectores.loc[(f_multisectores['VentaTotal'] >= 1000) & (f_multisectores['FOB'] >= 1000)]

      #Inserta "SUB"-dataframes al diccionario.
      if tipo==0 :
        #Extrae columnas RUC, PAIS, Tamaño, y agrupa por PAIS y Tamaño. Luego, cuenta empresas existentes por PAIS y Tamaño.
        multisectores_dict[f'{anio-i}'] = f_multisectores.loc[
            :,['NUM_DOCUMENTO', 'SECTOR_ES', f'Tamaño {anio-i}']
            ].groupby(
            ['SECTOR_ES', f'Tamaño {anio-i}'],as_index=False
            )['NUM_DOCUMENTO'].count()
        #Cambio de nombre Tamaño {anio-i} a 'Tamanio'
        #Importante: este nombre debe coincidir con el DF utilizado en el Merge final
        multisectores_dict[f'{anio-i}'].rename(
            columns={f'Tamaño {anio-i}':'Tamanio','NUM_DOCUMENTO':f'Empresas_{anio-i}'}, inplace=True)
      else :
        #Extrae columnas RUC y PAIS, y agrupa por PAIS. Luego, cuenta empresas existentes por PAIS.
        multisectores_dict[f'{anio-i}'] = f_multisectores.loc[
            :, ['NUM_DOCUMENTO', 'SECTOR_ES']
            ].groupby(
                ['SECTOR_ES'], as_index=False
                )['NUM_DOCUMENTO'].count()
        multisectores_dict[f'{anio-i}'].rename(
            columns={'NUM_DOCUMENTO':f'Empresas_{anio-i}_{mes}'}, inplace=True
            )

    #Construyendo el Merge Final.
    from itertools import product
    #Se crea la tupla 'sectores' (no modificable post creación)
    sectores= ("MINERIA TRADICIONAL", "AGRO TRADICIONAL", "PESCA TRADICIONAL",
               "PETROLEO Y GAS NATURAL", "AGROPECUARIO Y AGROINDUSTRIAS", "METAL - MECANICO",
               "QUIMICO", "PRENDAS DE VESTIR", "VARIOS", "TEXTIL",
               "MINERIA NO METALICA", "PESCA", "SIDERURGICO Y METALURGICO", "MADERAS"
               )
    if tipo==0 :
      #Se crea la tupla 'tamanos' (no modificable post creación)
      tamanos= (
          'Grande',
          'Mediana',
          'Pequeña',
          'Microempresa'
        )
      #Se crea el producto cartesiano entre sectores y tamanos.
      #Importante: la columna 'Tamanio' debe coincidir con el nombre del if tipo==0 del 1° for
      sxt=pd.DataFrame(product(sectores,tamanos),columns=['SECTOR_ES','Tamanio'])
      #Merge entre sxt y los "SUB" dataframes de multisectores_dict
      multisectores=pd.merge(sxt, multisectores_dict[f'{anio}'], how= 'outer', on=['SECTOR_ES','Tamanio'])
      for j in range (1,n):
        multisectores = pd.merge(
            multisectores,
            multisectores_dict[f'{anio-j}'],
            how= 'outer',
            on=['SECTOR_ES','Tamanio']
            )
    else :
      sxt=pd.DataFrame(sectores,columns=['SECTOR_ES'])
      multisectores=pd.merge(sxt, multisectores_dict[f'{anio}'], how= 'outer', on=['SECTOR_ES'])
      for j in range (1,n):
        multisectores = pd.merge(
            multisectores,
            multisectores_dict[f'{anio-j}'],
            how= 'outer',
            on=['SECTOR_ES']
            )

    #Exporta a Excel.
    multisectores.to_excel(f'multisectores_tipo{tipo}.xlsx', index=False)

In [ ]:
multisectores(df)

Ingrese el número de años a analizar, contando desde el argumento 'anio' (en este caso: 2023): 3
